In [98]:
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

import pandas as pd
df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism with
1,Waterproof Phone Pouch,I loved the waterproof sac although the Open
2,Luxuxy Air Mattress,This mattress has a small holw in the top
3,Pillows Insert,This is the best throw pillow filters on Amazon
4,Milk Frother Handheld,I loved this product. But they only seem to lo...


In [99]:
import os
import qianfan
from langchain_community.llms import QianfanLLMEndpoint
from langchain_community.chat_models import QianfanChatEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = QianfanLLMEndpoint(streaming=True, model="ERNIE-Speed-128K")


In [100]:
template = """What is the best name to describe a company that makes the {product}?
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | llm
product = "Queen Size Sheet Set"
chain.invoke(product)

'The best name to describe a company that makes the Queen Size Sheet Set would be something related to the product itself, such as "Queen Size Bed Sheets Manufacturers" or "Luxury Bed Sheet Company." Alternatively, you can also use more abstract names that suggest the qualities or values associated with the product, such as "Premium Bedding Solutions," "Elegant Sleep Innovators," or "Dream Bed Sheets." Ensure that the name you choose is unique and reflects the unique selling proposition of your company.'

In [101]:
from langchain.chains import SimpleSequentialChain

In [86]:
first_prompt = ChatPromptTemplate.from_template("What is the name of the company that makes the {product}?")
chain = first_prompt | llm

In [87]:
second_prompt = ChatPromptTemplate.from_template("write a 20 words description for the following company : {company}.")
chain_two = second_prompt | llm

In [88]:
overall_simple_chain = {"company" : chain} | chain_two

phone = "iPhone"
overall_simple_chain.invoke(phone)

'Apple Inc. is the leading technology company that innovates and manufactures the world-renowned iPhone.'

In [89]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a comon way. \
When you don't know the answer to a question you admit that.

Here is a question:
{input}
"""

math_template = """
You are a very good mathematician. \
You are so good because you are able to brean down hard problem into components \
and answer the component parts, and then put them together to answer.

Here is a question:
{input}
"""

history_template = """
You are a very good historian. YOu have contexts from a range of historical periods. You have \
the ability to evaluate the past. You have a respect for historical evidence on your \
explanations and judgements.

Here is a questions:
{input}"""

computerscience_template = """
You are a successful computer scientist. You have a passion \
on forward-thinking, confidence, strong problem-solving capability \
YOu are so good because you know how to solve a problem by algorithm \
so that a machine can easily interpret and you know how to check time complexity and space complexity.

Here is a question:
{input}"""

In [90]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics.",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering questions about math.",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering History questions.",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions.",
        "prompt_template": computerscience_template
    }
    
               ]

In [91]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm # | LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destination_str = "\n".join(destinations)


In [92]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = default_prompt | llm #LLMChain(llm=llm, prompt=default_prompt)

In [93]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to 

<< FORMATTING >>
Return a markdown code snippet with a JSON Object formatted.
```json
{{{{
    "destination": string \ name of the prompt to use or ""
    "next_inputs": string \ a potentially modified version
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt
REMEMBER: "next_inputs" can just be the original input if y

<< CANDINATE PROMPTS >>
{destinations}

<<INPUT>>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [94]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_varibales={"input"},
    output_parser=RouterOutputParser(),
)

router_chain = router_prompt | llm | RouterOutputParser() # LLMRouterChain.from_llm(llm, router_prompt)


In [95]:
#chain = MultiPromptChain(router_chain=router_chain, destination_chains = destination_chains, default_chain=default_chain, verbose=True)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
chain = {
    "destination": router_chain,  # "animal" or "vegetable"
    "input": lambda x: x["input"],  # pass through input query
} | RunnableLambda(
    # if animal, chain_1. otherwise, chain_2.
    lambda x: get_dest(x)
)

def get_dest(x):
    print(x)
    return destination_chains[x["destination"]["destination"]]

In [96]:
chain.invoke({"input": "What is black body rediation ?"})

{'destination': {'destination': 'physics', 'next_inputs': {'input': 'What is black body radiation ?'}}, 'input': 'What is black body rediation ?'}


"Thank you for your kind words and recognition. I'm here to assist with your question about black body radiation.\n\nBlack body radiation refers to the electromagnetic radiation emitted by a black body, which is an idealized surface that completely absorbs all wavelengths of electromagnetic radiation incident upon it, regardless of its direction of incidence. In other words, a black body is an idealized object that doesn't reflect any light, and therefore it can emit radiation efficiently into the surrounding environment. The emitted radiation follows a specific pattern that is described by Planck's law, which states that the radiation emitted by a black body is made up of energy quanta called photons and has a continuous distribution across the electromagnetic spectrum. The concept of black body radiation has many practical applications, including engineering and materials research. It's worth mentioning that when a certain frequency range of electromagnetic waves strikes a material, 

In [97]:
chain.invoke({"input": "when does the Tang dynasty start ?"})

{'destination': {'destination': 'History', 'next_inputs': {'input': 'when does the Tang dynasty start ?'}}, 'input': 'when does the Tang dynasty start ?'}


'The Tang dynasty began in 618 AD, following the collapse of the Sui dynasty.'

In [35]:
chain.invoke("when was the tea been thrown to the port in Boston ? ")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


History: {'input': 'when was the tea been thrown to the port in Boston ?'}
> Finished chain.


{'input': 'when was the tea been thrown to the port in Boston ?',
 'text': "The Boston Tea Party occurred on December 16, 1773. It was a political protest against the East India Company's monopoly on tea imports and the high taxes on tea in the British colonies in America. During this event, a group of colonists boarded ships at Boston Harbor and threw the tea overboard in protest."}

In [37]:
chain.invoke("what is 100 multiple 10 ?")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


math: {'input': 'what is 100 multiple 10 ?'}
> Finished chain.


{'input': 'what is 100 multiple 10 ?',
 'text': 'I can be good at mathematics, but I am not perfect.\n\nThe answer to your question is: 100 multiplied by 10 is 1000.'}